# Animation

In [1]:
import os
from skyfield.api import load
from ground_path import ground_path
from tqdm import tqdm
from subprocess import Popen, PIPE
import pandas as pd
from plotting import plot_ground_path
from ground_path import ground_path
import matplotlib.pyplot as plt
import cartopy
import matplotlib.gridspec as gridspec

### Load data

In [2]:
resource_url = 'http://celestrak.com/NORAD/elements/resource.txt'

fname = 'tle.txt'
if os.path.isfile(fname):
    os.remove(fname)

satellites = load.tle(resource_url, filename=fname, reload=True)
satellite = satellites['HODOYOSHI-1']
print(satellite)

[#################################] 100% tle.txt


EarthSatellite 'HODOYOSHI-1' number=40299 epoch=2018-08-06T18:53:38Z


In [3]:
# generate ground path
df = ground_path(satellite, '2018-08-01T00:00:00', '2018-08-02T00:00:00', '1min')
df.head()

100%|██████████| 1441/1441 [00:02<00:00, 611.47it/s]


,elevation,lon,lat,orbit,X,Y,Z
datetime,,,,,,,
2018-08-01 00:00:00,499.230718,-31.446707,64.790392,0,413.215323,-2909.273962,6198.663736
2018-08-01 00:01:00,499.578778,-34.793866,68.392579,0,222.727486,-2530.886132,6371.367487
2018-08-01 00:02:00,499.878571,-39.293342,71.914512,0,31.254193,-2141.301213,6515.803948
2018-08-01 00:03:00,500.119329,-45.714047,75.299764,0,-160.357438,-1742.243180,6631.333473
2018-08-01 00:04:00,500.292291,-55.537856,78.429194,0,-351.259742,-1335.477706,6717.444445


### Create frames

In [4]:
frames_directory = './frames'
if not os.path.exists(frames_directory):
    os.makedirs(frames_directory)

In [5]:
for i in tqdm(range(1, len(df), 5)):
    df_sub = df.iloc[:i]
    dt = df_sub.index[-1]
    
    title = "Hodoyoshi-1\n{:} ~ {:}".format(df_sub.index.min(), df_sub.index.max())

    fig = plt.figure(figsize=(20,10))
    # map projection
    projection0 = cartopy.crs.PlateCarree()
    # object transformation
    transform = cartopy.crs.PlateCarree()
    ax0 = fig.add_subplot(1,1,1, projection=projection0)
    plot_ground_path(ax0, df_sub, title, transform=transform, daynight=True, legend=True)
    plt.savefig(frames_directory + '/hodoyoshi1_{:%Y%m%d%H%M%S}.jpg'.format(dt))
    plt.cla()   # Clear axis
    plt.clf()   # Clear figure
    plt.close() # Close a figure window

100%|██████████| 288/288 [01:33<00:00,  3.09it/s]


### Create video

In [6]:
def make_video(in_dir, out_dir):
    p = Popen(['./make_video.sh "{:}" "{:}"'.format(in_dir, out_dir)], stdin=PIPE, stdout=PIPE, stderr=PIPE, shell=True)
    output, err = p.communicate()
    print(p.returncode)
    #print(output.decode('utf-8'), err.decode('utf-8'))
    del p
    return None

In [7]:
videos_directory = './videos'
if not os.path.exists(videos_directory):
    os.makedirs(videos_directory)

In [8]:
make_video(frames_directory + '/hodoyoshi1_*.jpg', videos_directory + '/hodoyoshi1.mp4')

0


In [9]:
%%HTML
<video controls>
  <source src="./videos/hodoyoshi1.mp4" type="video/mp4">
</video>